Create bash scripts

In [25]:
import os
from pathlib import Path
import numpy as np
from copy import deepcopy

from light_by_light.vacem_ini import W_to_E0, create_ini_file
from light_by_light.utils import write_yaml, read_yaml

In [12]:
template = '''#!/bin/bash
#SBATCH --job-name={jobname}
#SBATCH --partition=hij-gpu
#SBATCH --cpus-per-task={n_cpus}
#SBATCH --time=1-00:00:00
#SBATCH --mem=250GB
#SBATCH --output=/work/wi73yus/bash_output/{jobname}.%j.log
#SBATCH --error=/work/wi73yus/bash_output/{jobname}.%j.err

source ~/.bashrc  #  not automatically read by slurm
conda activate lbl  # activate here the anaconda environment you want to use

python {script_path} {script_kwargs}
'''

home_path = '/home/wi73yus/light-by-light'
work_path = '/work/wi73yus/light-by-light-experiments'

### Bash single simulation

In [20]:
# Laser pulse parameters
theta = 180
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 0,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
w0 = 8
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': w0*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 20,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'simulation'
geometry = 'z'
low_memory_mode = False
n_threads = 24
pol_idx = 0

# Save yaml file
data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx
}

folder = f'CCS'
jobname = f'theta_{theta}_zgeometry_w0_{w0}'
save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

# Create bash script
script_path = f'{home_path}/cluster/simulation_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template.format(jobname=jobname,
                       n_cpus=n_threads,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [21]:
default_name = 'default_params.yml'

default_yaml = f'{save_path}{default_name}'
yaml_data = read_yaml(default_yaml)
params = {}
lasers = yaml_data['lasers']
params['laser_params'] = [lasers[f'laser_{i}'] for i in range(len(lasers))]
params['save_path'] = save_path
keys = ['simbox_params', 'geometry', 'low_memory_mode']
for key in keys:
    params[key] = yaml_data[key]

create_ini_file(**params)

In [22]:
os.system(f'sbatch {bash_path}')

Submitted batch job 604291


0

### Bash script for 1d grid scan

In [7]:
# Laser pulse parameters
theta = 180
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 20,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'gridscan'
geometry = 'z'
low_memory_mode = False
n_threads = 24
pol_idx = 0

# Variable parameters 
laser_key = 'laser_1'
param_key = 'w0'
grid = [1,9,9,800e-9]
vary_params = {
    'lasers': {laser_key: {param_key: grid}}
}

# Save yaml file
data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx
}

folder = f'CCS'
jobname = f'{param_key}_theta_{theta}_zgeometry'
save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

vary_yaml = f'{save_path}vary_params.yml'
write_yaml(vary_yaml, vary_params)

# Create bash script
script_path = f'{home_path}/cluster/gridscan_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template.format(jobname=jobname,
                       n_cpus=n_threads,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [8]:
os.system(f'sbatch {bash_path}')

Submitted batch job 604225


0

### Bash script for optuna optimization

In [3]:
# Laser pulse parameters
theta = 180
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 4*800e-9,
    'W': 25,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 20,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'optuna'
geometry = 'xz'
low_memory_mode = False
n_threads = 24
pol_idx = 0

# Optuna params
obj_param = 'N_disc_num'
n_trials = 5

data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'obj_param': obj_param,
    'n_trials': n_trials
}

# Variable parameters 
laser_key = 'laser_1'
param_key = 'beta'
grid = [0,180,1,'uniform']
vary_params = {laser_key:
    {param_key: grid}
}

# Save yaml files
folder = f'CCS'
jobname = f'{param_key}_theta_{theta}'
save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

vary_yaml = f'{save_path}optuna_params.yml'
write_yaml(vary_yaml, vary_params)

script_path = f'{home_path}/cluster/optuna_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template.format(jobname=f'opt_{jobname}',
                       n_cpus=n_threads,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [4]:
os.system(f'sbatch {bash_path}')

Submitted batch job 600622


0

### Bash for numerical consistency check

In [39]:
# We need to test the same scattering configuration with a couple of 
# better numerical resolutions
# 1) increase time resolution,
# 2) increase spatial resolution
# 3) increase box size in time
# 4) increase box size in space

# Laser pulse parameters
theta = 180
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0x': 4*800e-9,
    'w0y': 1*800e-9,
    'W': 25,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_elliptical_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 20,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'gridscan'
geometry = 'xz'
low_memory_mode = False
n_threads = 12
pol_idx = 0

data = {
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
}

simbox_cases = {f'case_{i}': deepcopy(simbox_params) for i in range(0,5)}

simbox_cases['case_1']['resolutions']['t'] = 2
simbox_cases['case_2']['resolutions']['spatial'] = 3
simbox_cases['case_3']['box_factors']['t'] = 4
simbox_cases['case_4']['box_factors']['long'] = 10
simbox_cases['case_4']['box_factors']['trans'] = 25

folder = f'CCS'
jobname = f'theta_{theta}'
save_path = f'{work_path}/runs/resolution_check/{folder}/{jobname}/'

for key in simbox_cases.keys():
    data['simbox_params'] = simbox_cases[key]
    default_yaml = f'{save_path}{key}/default_params.yml'
    Path(os.path.dirname(default_yaml)).mkdir(parents=True, exist_ok=True)
    write_yaml(default_yaml, data)

In [41]:
template_res = '''#!/bin/bash
#SBATCH --job-name={jobname}
#SBATCH --partition=hij-gpu
#SBATCH --cpus-per-task={n_cpus}
#SBATCH --time=6:00:00
#SBATCH --mem=100GB
#SBATCH --output=/work/wi73yus/bash_output/{jobname}.%A_%a.log
#SBATCH --error=/work/wi73yus/bash_output/{jobname}.%A_%a.err
#SBATCH --array=0-4

arr=$SLURM_ARRAY_TASK_ID
echo $arr

source ~/.bashrc  #  not automatically read by slurm
conda activate lbl  # activate here the anaconda environment you want to use

python {script_path} {script_kwargs}
'''

script_path = f'{home_path}/cluster/simulation_cluster.py'
save_path_arr = save_path + 'case_$arr/'
script_kwargs = f'--save_path={save_path_arr}'

bash = template_res.format(jobname=f'numcheck_{jobname}',
                           n_cpus=n_threads,
                           script_path=script_path,
                           script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [42]:
os.system(f'sbatch {bash_path}')

Submitted batch job 600314


0

### Bash for 2d grid scan where one dimension is performed in parallel

### Bash for optuna optimization from several jobs

In [6]:
# Laser pulse parameters
theta = 180
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 4*800e-9,
    'W': 25,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 20,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'optuna'
geometry = 'xz'
low_memory_mode = False
n_threads = 24
pol_idx = 0

# Optuna params
obj_param = 'N_disc_num'
n_trials = 10

data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'obj_param': obj_param,
    'n_trials': n_trials
}

# Variable parameters 
laser_key = 'laser_1'
param_key = 'beta'
grid = [0,180,1,'uniform']
vary_params = {laser_key:
    {param_key: grid}
}

# Save yaml files
folder = f'CCS'
subfolder = f'{param_key}_theta_{theta}_multi'
save_path = f'{work_path}/runs/{mode}/{folder}/{subfolder}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

vary_yaml = f'{save_path}optuna_params.yml'
write_yaml(vary_yaml, vary_params)

In [8]:
template_optuna = '''#!/bin/bash
#SBATCH --job-name={jobname}
#SBATCH --partition=hij-gpu
#SBATCH --cpus-per-task={n_cpus}
#SBATCH --time=1-00:00:00
#SBATCH --mem=100GB
#SBATCH --output=/work/wi73yus/bash_output/{jobname}.%A_%a.log
#SBATCH --error=/work/wi73yus/bash_output/{jobname}.%A_%a.err
#SBATCH --array=0-2

source ~/.bashrc  #  not automatically read by slurm
conda activate lbl  # activate here the anaconda environment you want to use

echo $SLURM_ARRAY_TASK_ID

if [ $SLURM_ARRAY_TASK_ID -gt 0 ]
then
    sleep 60
fi

python {script_path} {script_kwargs}
'''

jobname = f'm_opt_{param_key}'

script_path = f'{home_path}/cluster/optuna_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template_optuna.format(jobname=f'{jobname}',
                              n_cpus=n_threads,
                              script_path=script_path,
                              script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [9]:
os.system(f'sbatch {bash_path}')

Submitted batch job 601630


0